In [63]:
import zipfile
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [64]:
URLS = {
    "transaction":"https://github.com/ThomasWarford/team_11/blob/main/transactions_1.csv.zip?raw=true", 
    "user": "https://github.com/ThomasWarford/team_11/blob/main/CUSTOMER_202304211626.csv.zip?raw=true",
    "product": "https://github.com/ThomasWarford/team_11/blob/main/PRODUCT_202304211040.csv.zip?raw=true",
    }


In [65]:
def one_hot_encode(df, column_name):
    one_hot = pd.get_dummies(df[column_name])
    df = df.drop(column_name, axis=1)
    df = df.join(one_hot)
    return df

In [66]:
df_trans = pd.read_csv(URLS["transaction"], compression='zip')
df_user = pd.read_csv(URLS["user"], compression='zip')
df_product = pd.read_csv(URLS["product"], compression='zip')
df_user.head()

,ID,TYPE,GENDER,DOB,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,POST_CODE,CITY,COUNTRY,POST_SUBSCRIBED,SMS_SUBSCRIBED,EMAIL_SUBSCRIBED,PHONE_SUBSCRIBED,SOCIAL_SUBSCRIBED,ANONYMISE
0,3e0698d7a4,New,Male,1988-04-16,Zelda,Schene,Zelda.SCHENE@yahoo.com,799947361,BH16,Liverpool,UK,True,True,True,True,True,True
1,3e06a74d39,Lookers,Male,2003-04-13,Betty,Asherman,Betty.ASHERMAN@yahoo.com,799947361,M1,Stockholm,Sweden,True,True,True,True,True,True
2,3e06b7960c,Loyal,Male,1972-04-20,Gusta,Leibowitz,Gusta.LEIBOWITZ@yahoo.com,799947361,BH16,Seattle,US,True,True,True,True,True,True
3,3e06ed2cfb,Need-based,Male,2002-04-13,Mona,Roca,Mona.ROCA@yahoo.com,799947361,BH16,Austin,US,True,True,True,True,True,True
4,3e07060e00,Loyal,Female,1998-04-14,Lesley,Bieck,Lesley.BIECK@yahoo.com,799947361,M1,Gothenburg,Sweden,True,True,True,True,True,True


In [67]:
df_2 = df_user.drop(["FIRST_NAME", "LAST_NAME", "EMAIL", "PHONE_NUMBER", "POST_CODE", "POST_SUBSCRIBED", "SMS_SUBSCRIBED", "SOCIAL_SUBSCRIBED", "CITY", "EMAIL_SUBSCRIBED", "PHONE_SUBSCRIBED", "ANONYMISE"], axis=1)
df_2 = one_hot_encode(df_2, "COUNTRY")
df_2 = one_hot_encode(df_2, "TYPE")
df_2 = one_hot_encode(df_2, "GENDER")
df_2 = df_2.drop(["Prefer not to say", "Bargain hunter"], 1)
df_3 = df_2.drop(["DOB", "ID"], 1)


df_3.head()

<ipython-input-67-805369f6b3c0>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_2 = df_2.drop(["Prefer not to say", "Bargain hunter"], 1)
<ipython-input-67-805369f6b3c0>:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_3 = df_2.drop(["DOB", "ID"], 1)


,Sweden,UK,US,Impulse,Lookers,Loyal,Need-based,New,Female,Male
0,0,1,0,0,0,0,0,1,0,1
1,1,0,0,0,1,0,0,0,0,1
2,0,0,1,0,0,1,0,0,0,1
3,0,0,1,0,0,0,1,0,0,1
4,1,0,0,0,0,1,0,0,1,0


In [68]:
kmeans = KMeans(n_clusters=6)
kmeans.fit(df_3)


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=6)

In [69]:
df_2["CLUSTER"] = kmeans.labels_
df_2.head()
# pd.plotting.scatter_matrix(df_3, c=df_2['CLUSTER'], figsize=(10,10), diagonal='hist')


,ID,DOB,Sweden,UK,US,Impulse,Lookers,Loyal,Need-based,New,Female,Male,CLUSTER
0,3e0698d7a4,1988-04-16,0,1,0,0,0,0,0,1,0,1,0
1,3e06a74d39,2003-04-13,1,0,0,0,1,0,0,0,0,1,3
2,3e06b7960c,1972-04-20,0,0,1,0,0,1,0,0,0,1,4
3,3e06ed2cfb,2002-04-13,0,0,1,0,0,0,1,0,0,1,4
4,3e07060e00,1998-04-14,1,0,0,0,0,1,0,0,1,0,2


In [70]:
df_2.head()

,ID,DOB,Sweden,UK,US,Impulse,Lookers,Loyal,Need-based,New,Female,Male,CLUSTER
0,3e0698d7a4,1988-04-16,0,1,0,0,0,0,0,1,0,1,0
1,3e06a74d39,2003-04-13,1,0,0,0,1,0,0,0,0,1,3
2,3e06b7960c,1972-04-20,0,0,1,0,0,1,0,0,0,1,4
3,3e06ed2cfb,2002-04-13,0,0,1,0,0,0,1,0,0,1,4
4,3e07060e00,1998-04-14,1,0,0,0,0,1,0,0,1,0,2


In [71]:
df_trans_new = df_trans.drop(['DISCOUNT','VALUE','QUANTITY','TYPE','SOURCE','PAYMENT_METHOD', "TRANSACTION_TIMESTAMP", "ID"],axis=1)
df_trans_new.PRODUCT_ID = df_trans_new.PRODUCT_ID.floordiv(1000)
to_join = df_2.filter(["ID", "CLUSTER"], axis=1)


df_trans_new.head()


,PRODUCT_ID,CUSTOMER_ID
0,611020,3926bb5245
1,779706,3926bb5245
2,695632,9a3a4ebf18
3,695632,9a3a4ebf18
4,812464,9a48a686f1


In [72]:
to_join.head()

,ID,CLUSTER
0,3e0698d7a4,0
1,3e06a74d39,3
2,3e06b7960c,4
3,3e06ed2cfb,4
4,3e07060e00,2


In [73]:
df_trans_new = df_trans_new.merge(to_join, how="left", right_on="ID" ,left_on="CUSTOMER_ID")
df_trans_new.head()
# df_trans_new = df_trans_new.drop("ID", axis=1)
df_trans_new.head()

,PRODUCT_ID,CUSTOMER_ID,ID,CLUSTER
0,611020,3926bb5245,3926bb5245,1
1,779706,3926bb5245,3926bb5245,1
2,695632,9a3a4ebf18,9a3a4ebf18,0
3,695632,9a3a4ebf18,9a3a4ebf18,0
4,812464,9a48a686f1,9a48a686f1,1


In [83]:
df_trans_new.head()
df_trans_new = df_trans_new.drop("ID", axis=1)


In [84]:
df_trans_new.head()

,PRODUCT_ID,CUSTOMER_ID,CLUSTER
0,611020,3926bb5245,1
1,779706,3926bb5245,1
2,695632,9a3a4ebf18,0
3,695632,9a3a4ebf18,0
4,812464,9a48a686f1,1


In [102]:
final = df_trans_new.groupby(["CLUSTER", "PRODUCT_ID"]).size().reset_index()
final.columns = ['cluster', 'product_id', 'count']
final
final.sort_values(by=["cluster", "count"], axis=0, ascending=False)

,cluster,product_id,count
154688,5,706016,6833
147341,5,599580,5359
147701,5,610776,4898
146107,5,562245,4653
155921,5,717490,3350
...,...,...,...
27760,0,941326,1
27772,0,942863,1
27781,0,946095,1
27785,0,946827,1


In [33]:
# df_trans.index
# df_user.head()
df_product.ID = df_product.ID.floordiv(1000)
df_product.head()




,ID,NAME,DESCRIPTION,AVAILABLE,LAST_AVAILABLE_DATE,PRICE,CATEGORY,SUB_CATEGORY,SIZE,COLOUR,GENDER,BRAND
0,735724,AK TIN TRAPEZE DRESS D3,"Wide, straight-cut dress in cotton jersey with...",True,2022-05-25,209,Garment Full body,Dress,L,Beige,Divided,AK Dresses & Outdoor
1,735724,AK TIN TRAPEZE DRESS D3,"Wide, straight-cut dress in cotton jersey with...",True,2022-05-25,42,Garment Full body,Dress,XS,Brown,Divided,AK Dresses & Outdoor
2,735724,AK TIN TRAPEZE DRESS D3,"Wide, straight-cut dress in cotton jersey with...",True,2022-05-25,170,Garment Full body,Dress,XS,Beige,Divided,AK Dresses & Outdoor
3,735724,AK TIN TRAPEZE DRESS D3,"Wide, straight-cut dress in cotton jersey with...",True,2022-05-25,289,Garment Full body,Dress,S,Black,Divided,AK Dresses & Outdoor
4,735724,AK TIN TRAPEZE DRESS,"Wide, straight-cut dress in cotton jersey with...",True,2022-05-25,217,Garment Full body,Dress,M,Mole,Divided,AK Dresses & Outdoor
